# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# connecting Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy' , 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)
    
# refence : https://www.tutorialspoint.com/cassandra/cassandra_create_keyspace .htm

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Creating Table and Insert statement for Querying artist, song title and song's length in the music app history that was heard based on Session and Item in that session

In [8]:
"""
created table with primary key as session and item
Table will provide information about artist , length of song and song title 
"""

query = "CREATE TABLE IF NOT EXISTS songs_item_session"
query = query + '''(
                    sessionId int,
                    itemInSession int,
                    artist text,
                    length float,
                    song text,
                    PRIMARY KEY (sessionId, itemInSession)
                    )'''
try:
    session.execute(query)
except Exception as e:
    print(e)       

In [9]:
"""
This funtion will insert records in songs_item_session table from event_datafile_new CSV file 
"""


file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                INSERT INTO songs_item_session( 
                sessionId, itemInSession, artist, length, song
                )"""
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0],  float(line[5]), line[9],))

In [10]:
"""
Select staetment to verify data
"""

query = """
        SELECT artist,
        song,
        length 
        FROM songs_item_session 
        WHERE sessionId = 338 AND itemInSession = 4
        """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


# convert data into dataframe and display top 5 possible rows
data = []    
for row in rows:
    data.append((row.artist, row.song,row.length))

df = pd.DataFrame(data,columns=['artist','song','length'])

df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Creating Table and Insert statement for Querying artist, song title and user first and last name in a given session and user 

In [11]:
"""
created table with Composite  key made of user and session with itemsession
Table will provide information about artist , song title with first and last name of a user 
"""

query = "CREATE TABLE IF NOT EXISTS songs_session"
query = query + '''(
                userId int,
                sessionId int, 
                itemInSession int,
                artist text,
                firstName  text,
                lastName  text,
                song text,
                PRIMARY KEY((userId, sessionId), itemInSession)
                )'''

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [12]:
"""
This funtion will insert records in songs_session table from event_datafile_new CSV file 
"""

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                INSERT INTO songs_session( 
                userId, sessionId, itemInSession, artist, firstName, lastName, song
                )"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

In [15]:
# Select
query = """
        SELECT artist,
        song,
        firstname,
        lastname 
        FROM songs_session 
        WHERE userId = 10 AND sessionId = 182
        """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# convert data into dataframe and display top 5 possible rows
data = [] 
for row in rows:
    data.append((row.artist, row.song,row.firstname, row.lastname))
df1 = pd.DataFrame(data,columns=['artist','song','firstname','lastname'])
df1.head()

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Creating Table and Insert statement for Querying First and last name of a user based on given song title 

In [16]:
"""
created table with PRIMARY key made of songs and user
Table will provide information about first and last name of a user 
"""

query = "CREATE TABLE IF NOT EXISTS user_listening_records"
query = query + '''(
                song text,
                userId int,
                firstName text,
                lastName text,
                PRIMARY KEY (song, userId)
                )'''
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
"""
This funtion will insert records in user_listening_records table from event_datafile_new CSV file 
"""

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                INSERT INTO user_listening_records (
                song, userId, firstName, lastName
                )"""
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
# select
query = """
        SELECT firstname,
        lastname 
        FROM user_listening_records 
        WHERE song = 'All Hands Against His Own'
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)   

# convert data into dataframe and display top 5 possible rows
data = []    
for row in rows:
    data.append((row.firstname, row.lastname))
df2 = pd.DataFrame(data,columns=['firstname','lastname'])
df2.head()

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the table before closing out the sessions

In [96]:
query = 'drop table songs_item_session'
try: 
    session.execute(query)
except Exception as e:
    print(e)

In [97]:
query = 'drop table songs_session'
try: 
    session.execute(query)
except Exception as e:
    print(e)

In [98]:
query = 'drop table user_listening_records'
try: 
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [99]:
session.shutdown()
cluster.shutdown()